In [2]:
import cv2
import matplotlib as plt
%matplotlib inline
import numpy as np

In [3]:
labels = open('coco.names.txt').read().strip().split('\n')#reading pretrained object labels

In [4]:
weights = 'yolov3.weights'#pretrained weights
architecture = 'yolov3.cfg.txt'#Neural network architecture

In [5]:
model = cv2.dnn.readNet(architecture,weights)#reading the neural network

In [6]:
colors = np.random.uniform(0, 255, size=(len(labels), 3))#bounding box color for each class

In [7]:
image = cv2.imread('street.jpg')#reading static image
Height,Width = image.shape[:2]#for scaling our bounding box wrt to image

In [8]:
#changes the image to required input format for neural network 
#adds the batch parameter, normalizes the pixels, resize to 416x416, swaps R and B channel
blob = cv2.dnn.blobFromImage(image, 1/255.0, (416, 416), swapRB=True, crop=False)
blob.shape

(1, 3, 416, 416)

In [9]:
model.setInput(blob)#Sets the new input value for the network

In [10]:
def forward_prop(model):
    layers = model.getLayerNames()#get all layer names
    output_layer = [layers[i[0]-1] for i in model.getUnconnectedOutLayers()]#get names of output layers
    return model.forward(output_layer)#does forward prop and gives output for the layers passed to it

In [11]:
CONFIDENCE = 0.65
IOU_THRESHOLD = 0.5
boxes, confidences, class_ids = [], [], []

In [12]:
outputs = forward_prop(model)#outputs of the last layer of NN 

In [13]:
for output in outputs:#loop through output of NN
    for pred in output:#loop through all predictions
        class_scores = pred[5:]#get class probs
        class_id = np.argmax(class_scores)#find max class prob
        class_prob = class_scores[class_id]#get class_id of class with max prob
        if class_prob>CONFIDENCE:
            #scale dimensions of bounding box wrt to image
            center_x = int(pred[0] * Width)
            center_y = int(pred[1] * Height)
            w = int(pred[2] * Width)
            h = int(pred[3] * Height)
            #getting top-left co-ordinates
            x = center_x - (w / 2)
            y = center_y - (h / 2)
            class_ids.append(class_id)
            confidences.append(float(class_prob))
            boxes.append([x, y, w, h])

In [14]:
indices=cv2.dnn.NMSBoxes(boxes,confidences,CONFIDENCE,IOU_THRESHOLD)#performs nms and returns boxes to be kept

In [15]:
def bounding_box(image,x,y,w,h,class_id,prob):
    label = labels[class_id]#getting label
    color = colors[class_id]#getting random color
    text = label+' : '+str(prob)
    cv2.rectangle(image,(x,y),(x+w,y+h),color=color,thickness=2)#bounding box rectangle
    (width,height)=cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, fontScale=1.3, thickness=1)[0]#getting size of text
    cv2.rectangle(image,(x,y-height),(x+width+2,y),color=color,thickness=-1)#box for placing text
    cv2.putText(image,text,(x,y-5),cv2.FONT_HERSHEY_SIMPLEX,fontScale=1,color=(0,0,0),thickness=1)#displaying text

In [16]:
for i in indices.flatten():
    x,y,w,h = boxes[i][0],boxes[i][1],boxes[i][2],boxes[i][3]
    prob=round(confidences[i], 3)
    bounding_box(image,int(x),int(y),int(w),int(h), class_ids[i] ,prob )

In [17]:
cv2.imshow("object detection", image)
cv2.waitKey()
cv2.destroyAllWindows()

In [18]:
cv2.imwrite('image.jpg',image)

True